In [6]:
# Importing Required Libraries
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor  
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import warnings
import pandas as pd
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"Dataset.csv")
df.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0


In [4]:
# Imputing the missing value of region record as the most common value
df['region'] = df['region'].fillna(2.)
df["region"] = df["region"].astype(int)
df['fpl_sel'] = df['fpl_sel'].replace('%','',regex=True).astype('float')
df.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.1,264,3,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.6,167,2,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.9,134,2,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.5,122,1,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.7,121,2,France,0,4,1,1,0


In [7]:
df['encoded_club'] = LabelEncoder().fit_transform(df['club'])
df['encoded_position'] = LabelEncoder().fit_transform(df['position'])
df['encoded_nationality'] = LabelEncoder().fit_transform(df['nationality'])

In [12]:
x = df[["page_views","fpl_value","fpl_sel","fpl_points","big_club"]]
y = df[["market_value"]]

In [13]:
# For Random Forest Regression
params = {
    'bootstrap': [True],
    'max_depth': [8, 10, 12, 14, 16],
    'min_samples_leaf': [1, 2, 3],
    'n_estimators': [40, 50, 60, 70],
    'max_features': [0.3, 0.4, 0.5]
}

forest_reg = GridSearchCV(RandomForestRegressor(), param_grid = params, scoring = 'r2', cv = 5)
forest_reg.fit(x,y)
acc_train = forest_reg.score(x, y)

print("Cross Validation score - " + str(forest_reg.best_score_))
print()

print("R^2 score of training data - " + str(acc_train))
print()
print(forest_reg.best_estimator_)
forest_best_params = forest_reg.best_params_

Cross Validation score - 0.688211742998476

R^2 score of training data - 0.9005377359222778

RandomForestRegressor(max_depth=12, max_features=0.3, min_samples_leaf=2,
                      n_estimators=60)


In [14]:
# fitting a random forest regressor
Forest_regr = RandomForestRegressor(max_depth=12, max_features=0.3,min_samples_leaf=2, n_estimators=60)
Forest_regr.fit(x, y)

RandomForestRegressor(max_depth=12, max_features=0.3, min_samples_leaf=2,
                      n_estimators=60)

In [15]:
import pickle

In [16]:
pickle.dump(Forest_regr,open("predictPrice.pkl","wb"))

In [17]:
def predict(page_views,	fpl_value,	fpl_sel,	fpl_points,	big_club):
    x = pd.DataFrame([])
    x[["page_views",	"fpl_value",	"fpl_sel",	"fpl_points",	"big_club"]] = [[page_views,	fpl_value,	fpl_sel,	fpl_points,	big_club]]
    model = pickle.load(open("predictPrice.pkl", "rb"))
    price = model.predict(x)
    return price[0]


In [18]:
predict(5.129899,	4.5	,0.8,	67	,0)

3.2959275076790533